In [1]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging
import pyarrow.parquet as pq
import warnings
import pyarrow as pa
from tqdm import tqdm
import torch
import os

2023-03-16 08:25:46.666219: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
os.environ["TOKENIZERS_PARALLELISM"] = "true"
warnings.filterwarnings("ignore", category=UserWarning)
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)
model_args = ClassificationArgs()
%env CUDA_VISIBLE_DEVICES = 4,5,6,7
model_args.n_gpu = 4
model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.no_cache = True
model_args.cache_dir = "cache_dir"
model_args.use_cached_eval_features = True
model_args.eval_batch_size = 16384
model_args.dataloader_num_workers = 32
cpu_num = 32 # 这里设置成你想运行的CPU个数
os.environ ['OMP_NUM_THREADS'] = str(cpu_num)
os.environ ['OPENBLAS_NUM_THREADS'] = str(cpu_num)
os.environ ['MKL_NUM_THREADS'] = str(cpu_num)
os.environ ['VECLIB_MAXIMUM_THREADS'] = str(cpu_num)
os.environ ['NUMEXPR_NUM_THREADS'] = str(cpu_num)
torch.set_num_threads(cpu_num)
torch.multiprocessing.set_sharing_strategy("file_system")

model = ClassificationModel(
    "xlmroberta",
    "/home/data/xuyijie/Documents/AllsidesScraper/news-title-bias/notebooks/02.Transformers/Models/outputs-xlmroberta-6gpu-batchsize384-20epoch_6label/checkpoint-11820-epoch-20",
    num_labels=6,
    args=model_args,
)
print("model loaded")

env: CUDA_VISIBLE_DEVICES=4,5,6,7
model loaded


In [2]:
twitter_content = pq.read_table(
    "/home/data/xuyijie/Documents/AllsidesScraper/news-title-bias/data/TwitterData/df_tweet_content_en_predictions_6kind.parquet").to_pandas()

In [18]:
twitter_content

,URL,Date,User,ReplyCount,RetweetCount,LikeCount,QuoteCount,ConversationID,RetweetedTweet,quotedTweet,inReplyToTweetID,inReplyToUser,mentionedUsers,hashtags,TweetContent,Language,Predictions
0,https://twitter.com/andya_d/status/15879267685...,2022-11-02 21:56:58+00:00,https://twitter.com/andya_d,1,0,1,0,1587725229396316160,None,None,1.5879161400340152e+18,https://twitter.com/yuroon,"[User(username='yuroon', id=29169337, displayn...",None,"Very, very nice.",en,5
1,https://twitter.com/andya_d/status/15875736234...,2022-11-01 22:33:42+00:00,https://twitter.com/andya_d,1,0,1,0,1587363839271739392,None,None,1.5875718109079675e+18,https://twitter.com/WeimarDay,"[User(username='WeimarDay', id=132273057754877...",None,No-show in sheer defiance? Rambo couldnt give ...,en,5
2,https://twitter.com/andya_d/status/15875189967...,2022-11-01 18:56:38+00:00,https://twitter.com/andya_d,0,0,2,0,1587363839271739392,None,None,1.5875182031142748e+18,https://twitter.com/WeimarDay,"[User(username='WeimarDay', id=132273057754877...",None,Grammar goofs. Toen (2007) al cheeky.,en,5
3,https://twitter.com/andya_d/status/15874439853...,2022-11-01 13:58:34+00:00,https://twitter.com/andya_d,0,0,0,0,1587096853858115584,None,None,1.5871065260992758e+18,https://twitter.com/ElliottArcand64,"[User(username='ElliottArcand64', id=696924884...",None,Elon must antivax it up against the WHO. That'...,en,5
4,https://twitter.com/andya_d/status/15872511012...,2022-11-01 01:12:06+00:00,https://twitter.com/andya_d,1,0,1,0,1586988562045599744,None,None,1.5872385848426086e+18,https://twitter.com/WeimarDay,"[User(username='WeimarDay', id=132273057754877...",None,Herum-erwappt.,en,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69499255,https://twitter.com/fetz_mv/status/10394790615...,2018-09-11 11:41:35+00:00,https://twitter.com/fetz_mv,0,0,0,0,1038027233519001600,None,None,1.0380272335190016e+18,https://twitter.com/realDonaldTrump,"[User(username='realDonaldTrump', id=25073877,...",None,Do you hear yourself?,en,5
69499256,https://twitter.com/fetz_mv/status/10391362625...,2018-09-10 12:59:25+00:00,https://twitter.com/fetz_mv,0,0,0,0,1039111866956566528,None,None,1.0391118669565665e+18,https://twitter.com/realDonaldTrump,"[User(username='realDonaldTrump', id=25073877,...",None,You are the joke and an embarrassment to the o...,en,5
69499257,https://twitter.com/fetz_mv/status/10374968680...,2018-09-06 00:25:03+00:00,https://twitter.com/fetz_mv,0,0,0,0,1037485664433070080,None,None,1.0374856644330701e+18,https://twitter.com/realDonaldTrump,"[User(username='realDonaldTrump', id=25073877,...",None,Sorry this is not a dictatorship. Free speech!!,en,5
69499258,https://twitter.com/fetz_mv/status/10374732683...,2018-09-05 22:51:17+00:00,https://twitter.com/fetz_mv,0,0,0,0,1037464177269514240,None,None,1.0374641772695142e+18,https://twitter.com/realDonaldTrump,"[User(username='realDonaldTrump', id=25073877,...",None,No! trying to save our country from you!,en,5
